In [2]:
import os
import openai
import json
import pandas as pd

In [3]:
openai.api_base = "http://localhost:1234/v1" # point to the local server
openai.api_key = "" # no need for an API key

In [4]:
with open('transcripts.json') as f:
  transcripts = json.load(f)

transcripts['1']

'Durante la visita, examiné a Angelina Thompson, una paciente de 74 años, que presentó una infección por hongos. Los síntomas incluían erupciones cutáneas anodales, parches dischromic y una erupción cutánea. Basado en los síntomas, confirmé que la paciente tenía una infección por hongos.\n\nPara prevenir la propagación adicional de la infección, le aconsejé a Angelina que se bañara dos veces al día, usara detol o neem en el agua del baño, mantuviera la zona infectada seca y usara ropa limpia. No receté ningún medicamento, ya que las precauciones deben ser suficientes para manejar la infección.\n\nEn general, la visita se centró en brindar orientación a la paciente sobre cómo manejar su infección por hongos a través de precauciones y medidas preventivas.'

In [5]:
test = pd.read_csv('test.csv')
test.head(13)

,Id,Transcript,Question
0,587d0feb-5780-43e1-9595-e19d4b31dc07,2055,What is the patient's name?
1,263e8884-e8ba-4266-bb0c-85271419a0b3,2055,What is the patient's age?
2,74c68eca-61b2-49d0-9b1c-0f6f886b04ff,2055,What is the patient's condition?
3,8572ab5d-f20a-4de5-ab44-f42b07e45a00,2055,What symptoms is the patient experiencing?
4,f5c92075-ef05-4fbf-a7a0-aa86c586ff02,2055,What precautions did the doctor advise?
5,03406fb0-e67d-4614-a745-ed02c7ac6c46,2055,What drug did the doctor prescribe?
6,1f0abde6-83ac-4e8f-b06f-9c9ff72beb88,291,What is the patient's name?
7,8ce89aa4-9d9c-4174-8e5e-957efbf41256,291,What is the patient's age?
8,08862b5d-dbc0-4ea1-884c-329eec085ff6,291,What is the patient's condition?
9,2f9bb4d6-8694-478e-a7f0-83ba534673de,291,What symptoms is the patient experiencing?


In [19]:
transcript = transcripts['3538']
transcript

'D: शुभ प्रभात, थॉमस जी। आज मुझे आपकी कैसे सहायता कर सकते हैं? \n\nP: शुभ प्रभात, डॉक्टर। मुझे अपने लम्बरों में कमजोरी, चक्कर आना और गले में दर्द के बारे में अनेक दिनों से तक समस्या है। \n\nD: महसूस हो रहा है, थॉमस जी। आपके लक्षणों और आपकी उम्र के आधार पर, मैं आशा कर रहा हूं कि आपको सर्विकल स्पॉन्डिलोसिस हो सकती है। \n\nP: यह क्या है, डॉक्टर? \n\nD: सर्विकल स्पॉन्डिलोसिस एक ऐसा स्थिति है जो आपके गले के हिस्से को प्रभावित करता है, जो आपके गले का हिस्सा है। यह 60 साल से ऊपर के लोगों में आम है और यह आपके गले में दर्द और स्टिफ़नस की वजह से हो सकता है, जैसे ही आपके लम्बरों में कमजोरी और नम्बर्स का महसूस हो सकता है। \n\nP: ओह, मैं समझ गया। क्या मुझे अच्छे से महसूस करने के लिए'

In [20]:
question = test[test['Transcript']==3538]
question

,Id,Transcript,Question
36,c1c3b8d3-3815-407b-a403-c33c9a8393d0,3538,What is the patient's name?
37,fa4021d6-1713-4e76-adbe-064480be6cff,3538,What is the patient's age?
38,c0ec170c-bde4-4e77-98f8-33855b134611,3538,What is the patient's condition?
39,8697aeb8-bb36-4171-b2bc-ee1f1804911b,3538,What symptoms is the patient experiencing?
40,b8497593-2536-468d-a260-070d1c1084f6,3538,What precautions did the doctor advise?
41,a6ecf101-3688-4852-b226-96b510934c62,3538,What drug did the doctor prescribe?


In [8]:
def chat_ans(transcript_cht, question_cht):
  response_temp = openai.ChatCompletion.create(
  model="local-model", # this field is currently unused
  messages=[
    {"role": "system", "content": "If you get transcript is not english, translate the script to english first and then answer the question. You are a doctor assistant to answer question with doctor note and conversation without makeup any information. You will answer all the question with information as short as possible. You HAVE to use the information I have provided. If you cannot find information from the text I provided, output N/A. If you get transcript other than english, translate the script to english first."},
    {"role": "user", "content": f"Now, DO NOT use any information above, Only use this transcript to answer question:### Input {transcript_cht}. The question is {question_cht}" },
  ],
  temperature = 0,
  max_tokens= 100,
  )
  return response_temp.choices[0].message.content

In [22]:
a = chat_ans(transcript_cht = transcript, question_cht = question.iloc[0,2])
a

"The patient's name is Shubh."

In [10]:
def pat_info(transcript_pat,question_pat):
  i=0
  temp_df = pd.DataFrame()
  for i in range(len(question_pat)):
    temp = chat_ans(transcript_cht = transcript_pat, question_cht= question_pat.iloc[i,2])
    temp_df.loc[len(temp_df),0] = question_pat.iloc[i,0]
    temp_df.loc[i,1] = temp
  return temp_df

In [11]:
test[test['Transcript']==2055]

,Id,Transcript,Question
0,587d0feb-5780-43e1-9595-e19d4b31dc07,2055,What is the patient's name?
1,263e8884-e8ba-4266-bb0c-85271419a0b3,2055,What is the patient's age?
2,74c68eca-61b2-49d0-9b1c-0f6f886b04ff,2055,What is the patient's condition?
3,8572ab5d-f20a-4de5-ab44-f42b07e45a00,2055,What symptoms is the patient experiencing?
4,f5c92075-ef05-4fbf-a7a0-aa86c586ff02,2055,What precautions did the doctor advise?
5,03406fb0-e67d-4614-a745-ed02c7ac6c46,2055,What drug did the doctor prescribe?


In [18]:
transcripts['3538']

'D: शुभ प्रभात, थॉमस जी। आज मुझे आपकी कैसे सहायता कर सकते हैं? \n\nP: शुभ प्रभात, डॉक्टर। मुझे अपने लम्बरों में कमजोरी, चक्कर आना और गले में दर्द के बारे में अनेक दिनों से तक समस्या है। \n\nD: महसूस हो रहा है, थॉमस जी। आपके लक्षणों और आपकी उम्र के आधार पर, मैं आशा कर रहा हूं कि आपको सर्विकल स्पॉन्डिलोसिस हो सकती है। \n\nP: यह क्या है, डॉक्टर? \n\nD: सर्विकल स्पॉन्डिलोसिस एक ऐसा स्थिति है जो आपके गले के हिस्से को प्रभावित करता है, जो आपके गले का हिस्सा है। यह 60 साल से ऊपर के लोगों में आम है और यह आपके गले में दर्द और स्टिफ़नस की वजह से हो सकता है, जैसे ही आपके लम्बरों में कमजोरी और नम्बर्स का महसूस हो सकता है। \n\nP: ओह, मैं समझ गया। क्या मुझे अच्छे से महसूस करने के लिए'

In [17]:
trans_id = test.iloc[39,1]
trans_id

3538

In [13]:
result_df = pd.DataFrame()
x=0

In [14]:
for x in range(9000,12006,6):
  trans_id = test.iloc[x,1]
  transcript_temp = transcripts[f'{trans_id}']
  question_temp = test[test['Transcript']==trans_id]
  result_temp = pat_info(transcript_pat= transcript_temp, question_pat= question_temp)
  result_df = result_df._append(result_temp)
  print(x)


9000
9006
9012
9018
9024
9030
9036
9042
9048
9054
9060
9066
9072
9078
9084
9090
9096
9102
9108
9114
9120
9126
9132
9138
9144
9150
9156
9162
9168
9174
9180
9186
9192
9198
9204
9210
9216
9222
9228
9234
9240
9246
9252
9258
9264
9270
9276
9282
9288
9294
9300
9306
9312
9318
9324
9330
9336
9342
9348
9354
9360
9366
9372
9378
9384
9390
9396
9402
9408
9414
9420
9426
9432
9438
9444
9450
9456
9462
9468
9474
9480
9486
9492
9498
9504
9510
9516
9522
9528
9534
9540
9546
9552
9558
9564
9570
9576
9582
9588
9594
9600
9606
9612
9618
9624
9630
9636
9642
9648
9654
9660
9666
9672
9678
9684
9690
9696
9702
9708
9714
9720
9726
9732
9738
9744
9750
9756
9762
9768
9774
9780
9786
9792
9798
9804
9810
9816
9822
9828
9834
9840
9846
9852
9858
9864
9870
9876
9882
9888
9894
9900
9906
9912
9918
9924
9930
9936
9942
9948
9954
9960
9966
9972
9978
9984
9990
9996
10002
10008
10014
10020
10026
10032
10038
10044
10050
10056
10062
10068
10074
10080
10086
10092
10098
10104
10110
10116
10122
10128
10134
10140
10146
10152
10158
101

In [36]:
result_df

,0,1
0,c6d724db-e267-4056-8f9c-9f2887886a65,The patient's name is Tracie Steele.
1,0e9ee56b-ed99-471b-9972-d37e3dfc860a,The patient's age is 70 years old.
2,54b9098d-0eeb-4804-9cf0-79ac61ed34da,The patient's condition is hepatitis A.
3,33825754-c807-475e-8099-2bd1b89650ca,"The patient, Tracie Steele, is experiencing sy..."
4,6a2e0284-bf47-4046-aafb-168f2b48ade6,The doctor advised Tracie Steele to consult th...
...,...,...
1,428968b3-e04b-4808-b71b-20ed93aaf0ed,The patient's age is not provided in the given...
2,3bd96a7e-b685-447a-8f23-75528b5a1b79,The patient has osteoarthritis.
3,82599e32-beff-460e-aa3f-b674fd9f7b4f,The patient is experiencing neck pain and swel...
4,528995c3-2f67-413d-bf46-ea138fdb4c62,The doctor advised Ms. Woods to monitor any ch...


In [15]:
finaldf = result_df

In [16]:
finaldf

,0,1
0,3ab8e08a-a37e-47d9-b62b-b8632a4ed020,The patient's name is Brian Williams.
1,a4f87db1-0a8a-41e0-a95e-1de592ea1135,The patient's age is 89 years old.
2,a5538a62-2c21-4559-9112-4bbf31f77cc4,The patient's condition is Pneumonia.
3,9795e804-730e-4e23-80f4-b819bdc2c902,"The patient, Mr. Brian Williams, is experienci..."
4,7beae5b4-9e6d-41c5-b5a4-40fab6820850,The doctor advised Mr. Williams to take medica...
...,...,...
1,89f03c30-3b65-4241-9157-9362f56381e6,The patient's age is not provided in the given...
2,25bf4a39-edd7-40d5-8c55-9ae5a6fdee08,"Based on the transcript provided, the patient'..."
3,774a32b8-237d-42f1-9907-d14d27d43858,"The patient is experiencing indigestion, inter..."
4,9fe29c8e-7189-4c47-ab27-c01d61cf39fc,The doctor advised Mr. Burns to avoid fatty an...
